## 二 醫學 NER model (BiLSTM-CRF)

我們本次的報告並沒有將醫電提供的資料進行重新建模，只是將目前現有的 NER 模型套用其中，最大的因素在於我們沒有太多時間將醫電提供的資料拆分成訓練 NER model 所需要的資料型態，在資料預處理上可說是曠日費時，但我們在讀過該篇方法後認為這或許是醫電公司未來可以考慮的方向，因此我們還是將連結與參考內容附上。

In [1]:
!pip install seqeval==0.0.5
!pip install keras==2.2.4 tensorflow==1.14.0 
!pip install h5py==2.10.0
!pip install opencc-python-reimplemented
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [3]:
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model,Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed, Dropout
from keras_contrib.layers.crf import CRF
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import re

In [4]:
!python --version
!nvcc --version

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## 1.1 建立 pretrain model 

本模型將醫學的 NER 分成五大類，分別是 TREATMENT (治療), BODY(身體部位) , SIGNS(症狀), CHECK(診斷), DISEASE (疾病)，並且將非屬於上述五類的文辭標記為 O ，另外本次的 pretrain model 作者一共使用了 600 筆的逐字稿進行訓練，算是滿小的資料集。

In [6]:
class LSTMNER:
    def __init__(self):
        self.train_path = '../input/medical-record-nlp-for-ner-task/train.txt'
        self.vocab_path = '../input/medical-record-nlp-for-ner-task/model/model/vocab.txt'
        self.embedding_file = '../input/medical-record-nlp-for-ner-task/model/model/token_vec_300.bin'
        self.model_path = '../input/medical-record-nlp-for-ner-task/model/model/tokenvec_bilstm2_crf_model_20.h5'
        self.word_dict = self.load_worddict()
        self.class_dict ={
                         'O':0,
                         'TREATMENT-I': 1,
                         'TREATMENT-B': 2,
                         'BODY-B': 3,
                         'BODY-I': 4,
                         'SIGNS-I': 5,
                         'SIGNS-B': 6,
                         'CHECK-B': 7,
                         'CHECK-I': 8,
                         'DISEASE-I': 9,
                         'DISEASE-B': 10
                        }
        self.label_dict = {j:i for i,j in self.class_dict.items()}
        self.EMBEDDING_DIM = 300
        self.EPOCHS = 10
        self.BATCH_SIZE = 128
        self.NUM_CLASSES = len(self.class_dict)
        self.VOCAB_SIZE = len(self.word_dict)
        self.TIME_STAMPS = 150
        self.embedding_matrix = self.build_embedding_matrix()
        self.model = self.tokenvec_bilstm2_crf_model()
        self.model.load_weights(self.model_path)

   
    def load_worddict(self):
        vocabs = [line.strip() for line in open(self.vocab_path)]
        word_dict = {wd: index for index, wd in enumerate(vocabs)}
        return word_dict

  
    def build_input(self, text):
        x = []
        for char in text:
            if char not in self.word_dict:
                char = 'UNK'
            x.append(self.word_dict.get(char))
        x = pad_sequences([x], self.TIME_STAMPS)
        return x

    def predict(self, text, prin = True):
        str = self.build_input(text)
        raw = self.model.predict(str)[0][-self.TIME_STAMPS:]
        result = [np.argmax(row) for row in raw]
        chars = [i for i in text]
        tags = [self.label_dict[i] for i in result][len(result)-len(text):]
        res = list(zip(chars, tags))
        if prin:
            print(res)
        return res

    
    def load_pretrained_embedding(self):
        embeddings_dict = {}
        with open(self.embedding_file, 'r') as f:
            for line in f:
                values = line.strip().split(' ')
                if len(values) < 300:
                    continue
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_dict[word] = coefs
        print('Found %s word vectors.' % len(embeddings_dict))
        return embeddings_dict

    
    def build_embedding_matrix(self):
        embedding_dict = self.load_pretrained_embedding()
        embedding_matrix = np.zeros((self.VOCAB_SIZE + 1, self.EMBEDDING_DIM))
        for word, i in self.word_dict.items():
            embedding_vector = embedding_dict.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

        return embedding_matrix

    
    def tokenvec_bilstm2_crf_model(self):
        model = Sequential()
        embedding_layer = Embedding(self.VOCAB_SIZE + 1,
                                    self.EMBEDDING_DIM,
                                    weights=[self.embedding_matrix],
                                    input_length=self.TIME_STAMPS,
                                    trainable=False,
                                    mask_zero=True)
        model.add(embedding_layer)
        model.add(Bidirectional(LSTM(128, return_sequences=True)))
        model.add(Dropout(0.5))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Dropout(0.5))
        model.add(TimeDistributed(Dense(self.NUM_CLASSES)))
        crf_layer = CRF(self.NUM_CLASSES, sparse_target=True)
        model.add(crf_layer)
        model.compile('adam', loss=crf_layer.loss_function, metrics=[crf_layer.accuracy])
        model.summary()
        return model

In [7]:
ner = LSTMNER()

## 1.2 測試 pretrain model

In [8]:
s = input('enter an sent:').strip()
ner.predict(s)

我們發現該模型將頭痛視為是兩個不同的 NER，分別是診斷與症狀，但出現該結果最大的原因可能在於原作者訓練模型時採用的是簡體中文，但我們輸入的是繁體中文，因此我們將繁體改成簡體後再試一次。 

In [9]:
from opencc import OpenCC
cc = OpenCC('t2s')
s = input('enter an sent:').strip()
ner.predict(cc.convert(s))

這次的結果比較符合我們的預期，因為頭痛應當是一個字詞代表著症狀。

## 1.3 將醫電的資料帶入模型

In [23]:
metadata = pd.read_csv('../input/medical-record-nlp-for-ner-task/MedData.csv',encoding = 'utf-8')
metadata.head()

我們一共擷取了 96 筆資料，並且每筆資料拜代表著一筆護理師的逐字稿與交班表的內容，其中 raw data 是逐字稿的全文，而 process 是交班表中的入院經過，之所以特別挑入院經過是因為我們有朋友是護理人員，她說入院經過跟生命徵象是交班表中比較重要的內容，但是滿多的逐字稿都沒有詳細的生命徵象，因此我們將入院經過抓出，希望透過針對 raw data 進行 NER 將 NER 為 sign 的內容抓出，而 sign 的內容基本上就是該病患的入院經過，因此我們特別只抓入院經過是希望能跟經過 NER model 後的結果進行比對。 

分析的步驟其實很簡單如下: 
1. 將中文以外的部分去除 (但這個部分其實不是很好，因為很多護理人員都會中英文混雜)
2. 將繁體中文轉成檢體後丟入 pretrain 的 ner model 
3. 抓出有識別為 sign or disease 的部份

In [24]:
metadata['raw_data'] = metadata.raw_data.apply(lambda x: re.sub('[^\u4e00-\u9fa5]+', '', x))
metadata['medner'] = metadata.raw_data.apply(lambda x: ner.predict(cc.convert(x),prin=False))
metadata.head()

In [25]:
metadata['medsigns'] = metadata.medner.apply(lambda x: [t for t in x if bool(re.search('SIGN|DISEASE',t[1]))] )
metadata2 = metadata[[bool(i) for i in metadata['medsigns']]]
metadata2

我們比對 medsign 跟 process 兩個欄位的結果發現模型分類的效果其實很差，幾乎都沒有把 sign 給抓出來，我們認為可能有以下問題: 
1. 原模型在訓練的時後所採用的資料筆數過少
2. 原模型在訓練的時候所採用的資料跟我們拿來預測的資料結構差異過大
3. 原模型的訓練資料內容較精簡，但實際的預測資料反而過於冗長

因此我們將 process 的內容丟入模型當中看看結果如何，原因在於 process 的內容較精簡，我們猜效果可能會比將整段的 raw data 丟入還要更好。

In [28]:
metadata['process'] = metadata.process.apply(lambda x: re.sub('[^\u4e00-\u9fa5]+', '', x))
metadata['process_medner'] = metadata.process.apply(lambda x: ner.predict(cc.convert(x),prin=False))
metadata['processsign'] = metadata.process_medner.apply(lambda x: [t for t in x if bool(re.search('SIGN|DISEASE',t[1]))] )
metadata3 = metadata[[bool(i) for i in metadata['processsign']]]
metadata3[['process','processsign']]

## **小節**
由以上的嘗試我們得知，當我們只針對 process 進行 NER 預測時效果是還不錯的，基本上症狀跟疾病都有被分類出來，但我們將 raw data 整筆丟入時卻沒有辦法將其中的疾病或是症狀分類出來，我們認為最大的原因在於 raw data 包含太多沒有用到的資訊，以及 raw data 跟原模型在訓練時使用的資料長度相比差異太多，最後是我們認為中國在語法的使用上可能還是跟我們在文字的使用上有差異，導致我們無法直接將預訓練的模型直接套用。但該模型的生成方是我們認為是醫電公司可以參考且實際將資料進行分類、處理以及建模，應當能有不錯的成效。